#Workshop Notebook

This jupyter notebook will make for the interactive part of this workshop

## Step 1: Inspect the data

Usually, the first thing we want to do when dealing with any new type of data, we want to inspect it first to get some intuitions for it. By visualizing the data, we can often get some ideas as to how to tackle the data and features we can extract from it.

In [ ]:
from keras.layers import Dense, Conv1D, MaxPool1D, Flatten
from keras.models import Sequential
from scipy.signal import periodogram

import numpy as np
import matplotlib.pyplot as plt
from helpers import load_binary
from keras.utils import to_categorical

%matplotlib inline

# model = Sequential()
# model.add(Dense(2, input_dim=1))
# model.add(Dense(1))
# model.compile(optimizer='adam', loss='mse')

with open('data/labels.csv', 'r') as fp:
    labels = [line.replace('\n', '').split(',') for line in fp.readlines()[1:]]

np.random.seed(1337)
np.random.shuffle(labels)

x_train, y_train = [], []
for label in labels:
    filepath = 'data/signals/%s' % label[0]
    label_id = 4 if label[1] == '' else int(label[1])
    label_id = to_categorical(label_id, 5)
    data = load_binary(filepath)
    _, data = periodogram(data, fs=2000)
    x_train.append(data)
    y_train.append(label_id)
x_train = np.array(x_train).reshape((len(x_train), -1, 1))
y_train = np.array(y_train)

model = Sequential()
model.add(Conv1D(32, 9, padding='valid', input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(MaxPool1D(2))
model.add(Conv1D(64, 9))
model.add(MaxPool1D(2))
model.add(Conv1D(128, 9))
model.add(Flatten())
model.add(Dense(5, activation='softmax'))
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

logger = model.fit(x_train, y_train, epochs=200, batch_size=32, validation_split=0.25)
model.evaluate(x_train[-25:], y_train[-25:])

plt.plot(logger.history['accuracy'])
plt.plot(logger.history['val_accuracy'])
plt.tight_layout()
plt.show()

# data = np.array(x[1337]).reshape((1, 1))
# p = model.predict(data)

# print(p)

# plt.plot(x)
# plt.tight_layout()
# plt.show()
